Настоящее время в отрасли используется подход к реализации отдельных уровней, включающий **графический** и **операторный** уровни. **Графический уровень** отвечает за объединение или перегруппировку вычислительного графа, а **операторный уровень** отвечает за компиляцию объединенных или перегруппированных операторов в высокопроизводительные исполняемые операторы. Графический уровень обычно обрабатывается и оптимизируется с помощью высокоуровневого IR-анализа на основе тензора, в то время как операторный уровень анализируется и оптимизируется с помощью низкоуровневого IR-анализа на основе вычислительных команд. Этот искусственный процесс, основанный на разделении уровней, значительно увеличивает сложность выполнения совместной оптимизации как на графическом, так и на вычислительном уровнях.

MindSpore применяет технологию слияния графов и ядра для более эффективного решения этой проблемы. Типичные сети в различных категориях, таких как NLP и recommendation, демонстрируют значительный прирост скорости обучения после включения Graph-Kernel Fusion Acceleration Engine. Одной из основных причин является наличие большого количества небольших комбинаций операторов в этих сетях, которые имеют больше возможностей для оптимизации.

## Architecture and Overall Process
Основная идея на уровне графа заключается в том, чтобы активировать составной оператор, затем выполнить агрегацию через границы и оптимизацию, а в конце выполнить разбиение оператора ядра. 

Основные шаги включают:
1) Расширене составного оператора: Из базовых операторов созданется составной оператор и обозначается, как базовый. Формируется составной подграф для упрощения последующей оптимизации через границы и разбиения оператора.
2) Агрегация операторов: Агрегация смежных элементарных операторов или составных подграфов для формирования более крупных агрегированных подграфов для последующей оптимизации через границы и разбиения оператора.
3) Высокоуровневая оптимизация: На основе агрегированных подграфов, полученных на предыдущих шагах, можно выполнить множество оптимизаций через границы, таких как алгебраическое упрощение и извлечение общих подвыражений (CSE).
4) Разделения ядра: Исходя из вычислительных характеристик и производительности оператора слияния, выполняется разбиение оператора на агрегированном вычислительном подграфе.

Затем оптимизированный вычислительный граф передается в MindSpore AKG в качестве подграфа для дальнейшей бэкенд-оптимизации и генерации целевого кода.

![graph](./pictures/GKFAE-overall.png)

## Fusion Operator Acceleration Optimization (MindSpore AKG)

Оптимизация слияния графических ядер может обеспечить экспоненциальное улучшение производительности. Однако с увеличением возможностей слияния графических ядер разработка операторов слияния становится узким местом для дальнейшего улучшения этих возможностей. Технология автоматической генерации операторов слияния может решить проблему высокого порога программирования для разработки операторов слияния на основе DSA (Domain Specific Architecture), позволяя программистам сосредоточиться на логике реализации операторов во время их разработки, не углубляясь в бэкенд-оптимизацию, что значительно повышает эффективность разработки. Особенно важны такие технологии для сценариев с комплексной архитектурой аппаратного обеспечения и сложными операторами и операторами слияния.

Таким образом, MindSpore AKG ускоряет оптимизацию и автоматическую генерацию операторов слияния на основе технологии полигональной компиляции (Polyhedral Model), что помогает операторам слияния, оптимизированным модулем слияния графических ядер MindSpore, автоматически генерировать высокопроизводительные ядра на гетерогенных аппаратных платформах (GPU/Ascend) и улучшать производительность обучения в MindSpore.


### Overall Structure

1) **Нормализация IR**. На вход MindSpore AKG подается объединённый подграф, оптимизированный модулем слияния графов MindSpore. Операторы в этом подграфе выражаются с помощью различных описаний, таких как Compute/IR Builder/Hybrid в TVM. Затем DSL преобразуется в Halide IR (Halide — это язык, используемый для разработки высокопроизводительных приложений для обработки изображений и вычислений с массивами, который может использоваться в качестве промежуточного представления для разделения алгоритмов и оптимизаций) и производится нормализация IR. После начального упрощения и оптимизации Halide IR преобразуется в дерево планирования, необходимое для модуля Poly.

2) **Оптимизация планирования в модуле Poly**. Используя алгоритм планирования Pluto на основе полиэдральной технологии, осуществляется автоматическое слияние циклов, автоматическое переупорядочивание и другие преобразования для автоматической генерации начального плана, удовлетворяющего требованиям параллелизма и локальности данных для объединенного оператора. <br>
Для быстрой адаптации к различным аппаратным платформам этапы оптимизации в модуле Poly делятся на аппаратно-независимые общие оптимизации и аппаратно-зависимые специфические оптимизации, которые объединяются и комбинируются в зависимости от характеристик оборудования во время компиляции, чтобы обеспечить быструю адаптацию к гетерогенным аппаратным платформам. Такие этапы, как автоматическая нарезка, автоматическое отображение и автоматическое усиление памяти, будут выполнять различные оптимизации в зависимости от особенностей архитектуры оборудования.

3) **Оптимизация на стороне платформ**. Для дальнейшего повышения производительности операторов разработаны определенные этапы оптимизации для различных аппаратных платформ, такие как выравнивание данных и сопоставление инструкций для платформы Ascend, векторизованный доступ и вставка инструкций синхронизации для платформы GPU, и, наконец, генерируется код, соответствующий конкретной платформе.

**Замечание:**

**Polyhedral Model** — это математический подход для представления и оптимизации вычислительных программ, особенно тех, которые включают циклы и многомерные массивы. Он широко применяется в компиляторах для анализа и трансформации программ, работающих с большими объемами данных и параллельными вычислениями, особенно для таких архитектур, как GPU или многопроцессорные системы.

**Polyhedral Model** — это распространенный метод циклической вложенной оптимизации в области компьютерно-компиляторной оптимизации, теоретической основой которого является арифметика Пресбургера. Многогранная модель позволяет нам анализировать зависимости между чтением и записью инструкций в программе, а затем обеспечивает теоретическую поддержку для последующих циклических преобразований. Ядром циклической оптимизации многогранной модели является алгоритм планирования, который может определять цели оптимизации на основе характеристик аппаратной архитектуры (таких как параллелизм и локальность данных) и преобразовывать задачу циклической оптимизации в задачу целочисленного программирования для решения. В MindSpore AKG планировщик ISL, основанный на целочисленном линейном программировании, в основном используется для выполнения нового преобразования расписания во входной программе. В планировщике задач преобладает алгоритм Pluto, который дополняется алгоритмом Feautrier, который обеспечивает оптимальное соотношение между параллелизмом программ и локальностью. (MindSpore Doc)

**Pluto scheduling algorithm** — это алгоритм, основанный на полиэдральной модели, который используется для автоматического преобразования циклов в программах с целью оптимизации производительности.

**Halide:** https://halide-lang.org/



## Auto-Tiling

**Tiling** — это метод оптимизации циклов в программировании, особенно в высокопроизводительных вычислениях, который улучшает доступ к данным и ускоряет выполнение программы.

Когда программа выполняет циклы, особенно с большими массивами данных, она может обращаться к данным, которые находятся далеко друг от друга в памяти. Это приводит к большим затратам на операции с памятью, так как данные не помещаются в быстрый кэш процессора, и программа вынуждена постоянно загружать данные из медленной основной памяти.

Tiling решает эту проблему, разбивая большие циклы на более мелкие "плитки" (или блоки) меньшего размера, которые хорошо помещаются в кэш процессора. Вместо того чтобы перебирать все элементы большого массива по очереди, программа сначала обрабатывает небольшой блок данных, которые могут быть загружены в кэш и быстро обработаны. Как только этот блок обработан, загружается и обрабатывается следующий блок

MindSpore AKG предоставляет модуль автоматического разбиения на блоки, основной процесс включает:

1) Анализ входных данных. Авторазбиение преобразует логику входного оператора в дерево планирования с помощью модуля Poly, и модель осевого пространства извлекает это дерево для анализа и выделяет всё осевое пространство, которое можно разбить на блоки и запланировать.

2) Обрезка осевого пространства. Авторазбиение использует две модели генерации ограничений для анализа логики оператора и информации об оборудовании, чтобы сгенерировать соответствующие ограничения. Эти ограничения делятся на жёсткие и мягкие в зависимости от области действия оператора, что приводит к сокращению осевого пространства до ограниченного осевого пространства.

3) Решение осевого пространства. Авторазбиение генерирует различные функции оптимизации на основе информации об операторе и оборудовании. Модель решения находит уникальное решение в ограниченном осевом пространстве для генерации итоговой конфигурации планирования разбиения на блоки

## Auto-mapping

**Auto-mapping** относится к автоматическому сопоставлению данных и экземпляров в порядке выполнения с многопоточными процессорами, такими как блок потоков и поток графического процессора, на аппаратном сервере многопоточной архитектуры. С помощью функции автоматического картографирования мы можем:

1) Reduce code complexity
Как показано на рисунке ниже, с фигурой оператора 8 * 12, функция Auto-Tiling попытается взять мозаику в пункте (a), чтобы уменьшить суждения о круглых границах, показанные синим цветом в пункте (b)

![Reduce](./pictures/Auto-mapping-reduce.png)

Затем функция Auto-Mapping также пытается выделить размер потока, который можно разделить на разбитые данные, чтобы улучшить использование потоков, как в следующем примере с потоком, равным 4.

![Threads](./pictures/Auto-mapping-threads.png.png)

2) Optimize Block/Thread/Tile ratio

Auto-Mapping учитывает аппаратные характеристики графического процессора при назначении размера блока, размера потока и размера плитки. Оптимизация производительности выполняется путем корректировки соотношения трех факторов для улучшения трех аспектов: использования, пропускной способности памяти и скорости доступа.